Up to know a single agent entered the game but what if we would like to set some diversity? This is particularly relevant when creative processes or opinions enter the arena:

* We may want to do a SEO analysis
* Create a report on the progress of an employee
* Define a hybrid work policy

And all these tasks **will need human expertise** but we can have it started with a nice draft bringing different opinions. Something that would take several workshops and involve different roles, all automated. The idea is definitely captivating.

## MultiAgent frameworks

[CrewAI](https://www.crewai.com/) was created with this idea in mind, to programmatically be able to define a crowd, give them a task and allow them to discuss, evolve and refine the outcome before presenting it to the user.

Similarly, [Agno](https://app.agno.com/) offers the ability to deploy and test multiagent applications thanks to its playground.

![agnomulti](../images/agno_multi.png)

We can stick to LangChain if we would like to have a higher control of the architecture being implemented.

![](https://langchain-ai.github.io/langgraph/concepts/img/multi_agent/architectures.png)

More on https://langchain-ai.github.io/langgraph/concepts/multi_agent/

But the main point of previous frameworks is that it makes it easier to define the handover of tasks between agents and the overall interaction between the agents before a final response is provided.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

We will need to equip our researchers with some tools, web search tools in particular. These tools can be customized as much as we might need to narrow down the potential responses.

In [2]:
from agno.tools.tavily import TavilyTools

# Search engine
search = TavilyTools(
    include_answer=True,
    search_depth='basic',
    format='markdown',
    max_tokens=100
)

A first agent will take care of the SEO analysis while the second agent optimizes the content. If both collaborate the impact of the content creating will be given by the ability to search and refine the context by the first agent, the SEO analyst looking for keywords.

In [5]:
from agno.agent import Agent
from agno.models.google import Gemini

seo_analyst = Agent(
    name="SEO Analyst",
    role="Discovers the keywords needed to boost the position of the site to be improved.",
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    tools=[search],
    instructions="""
        Backstory:
        
        You work in Spain, most of your requests come from Spanish companies so you prioritize the positioning in Spanish speaking searches.
        Your specialty is identifying keywords to better position the products highlighted by the company.
        You conduct research on competitors and search results to better differentiate the offering and be more competitive.

        Response format:
        Always include sources.
    """,
)

strategist = Agent(
    name="Content strategist",
    role="Creates attractive and engaging content to improve SEO positioning.",
    model=Gemini(id="gemini-2.5-flash", temperature=0.6),
    tools=[search], # It might not need it but just in case
    instructions="""
        Backstory:
        
        You are a renowned Content Strategist, known for your insightful and engaging articles.
        Your specialty is transforming complex concepts into captivating narratives with high SEO impact.

        Response format:
        Always include sources.
    """,
)

We need to team the up and assign them the task to be performed. This requires a **manager** with some reasoning capability to efficiently use the resources it has. It then requires:

* Reasoning capabilities
* Some instructions
* when to end the work

In [9]:
from agno.team.team import Team
from agno.tools.reasoning import ReasoningTools

seon_team = Team(
    name="SEO improving content creation team",
    model=Gemini(id="gemini-2.5-flash", temperature=0.3),
    members=[seo_analyst, strategist],
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Collaborate to provide comprehensive SEO improving insights",
        "Consider both fundamental analysis and market sentiment",
        "Use an engaging language to provide guidance to the user asking the questions",
        "Present findings in a structured, easy-to-follow format",
        "Only output the final consolidated response, not individual agent responses",
    ],
    markdown=True,
    show_members_responses=True,
    enable_agentic_state=True,
)

In [10]:
seon_team.print_response(
"""The Bridge School is providing upskiller courses dedicated to improving the abilities of professionals with a special focus on Data Science and Artificial intelligence.
How would you recommend improving the text and visuals of that section fo the site to achieve that goal?
""",
    stream=True,
    show_full_reasoning=True,
    stream_intermediate_steps=True,
)

Output()